In [6]:
import re
import collections
import json

import numpy as np
import xml2json
import sklearn
import pymorphy2

from sklearn import svm

In [7]:
b = json.loads(open('train.json').read())

In [8]:
list(b)
b = b['pma_xml_export']
b = b['database']
b = b['table']

In [9]:
morph = pymorphy2.MorphAnalyzer()
dictTrain = collections.Counter()

In [10]:
trainMatr = {}

In [11]:
for tweetText in b:
    for word in re.findall(r'[а-яА-Я]+',tweetText['column'][3]['#text']):
#        add_to_dict(diccc, word.lower())
        dictTrain[morph.parse(word.lower())[0].normal_form]+=1

In [12]:
str(dictTrain)[:]
itr = 0
for wordd in dictTrain:
    if dictTrain[wordd] > 50: 
        itr+=1
        trainMatr[wordd] = itr
itr

182

In [13]:
#функция isbank() возвращает вектор, чьи компоненты ==0, кроме той, которая соотв. номеру банка 1. сбербанк, 2. втб, 3. газпром, 4. альфабанк 5. банкмосквы, 6. райффайзен, 7. уралсиб, 8. россельхоз

In [14]:
def isbank(c,i):
    a = [0 for i in range(8)]
    for j in range(8):
        if c[i]['column'][j+4]['#text'] != 'NULL':
            a[j] = 1
    return a

In [15]:
def gety(c,i):
    y = 0
    for j in range(8):
        if c[i]['column'][j+4]['#text'] != 'NULL':
           
            y = float(c[i]['column'][j+4]['#text'])
    return y

In [18]:
lTrain = list()
lY = list()
ind = 0
for tweetText in b:
    vectTrain = [0 for i in range(itr+8)]
    for wordd in re.findall(r'[а-яА-Я]+',tweetText['column'][3]['#text']):
        if wordd in trainMatr:            
            vectTrain[trainMatr[wordd]] +=1
            #vordvec.append(freqmatr[word])
    vectTrain[itr:] = isbank(b, ind)
    lY.append(gety(b,ind))
    ind+=1
    lTrain.append(vectTrain)

In [19]:
X_train, Y_train = np.asarray(lTrain), np.asarray(lY)
print(X_train.shape, Y_train.shape)

(9392, 190) (9392,)


In [20]:
clf = sklearn.svm.SVC()
#clf = GradientBoostingClassifier(n_estimators=10000)
clf.fit(X_train[100:], Y_train[100:])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [21]:
print((clf.predict(X_train[:100]) == Y_train[:100]).sum())

74
